# Demo2

## Train a new model

In [1]:
from DeezyMatch import train as dm_train

# train a new model
dm_train(input_file_path="./inputs/input_dfm_demo2.yaml", 
         dataset_path="../../dataset/BL_IAMS_geonames.tsv",
         model_name="BL_model")

2020-12-07 09:37:14 MAC-ATI0414 [INFO] read input file: ./inputs/input_dfm_demo2.yaml
2020-12-07 09:37:14 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:37:14 MAC-ATI0414 [INFO] pytorch will use: cpu
2020-12-07 09:37:14 MAC-ATI0414 [INFO] read CSV file: ../../dataset/BL_IAMS_geonames.tsv
2020-12-07 09:37:14 MAC-ATI0414 [INFO] number of labels, True: 9532 and False: 9135
2020-12-07 09:37:14 MAC-ATI0414 [INFO] Splitting the Dataset
2020-12-07 09:37:14 MAC-ATI0414 [INFO] finish splitting the Dataset. User time: 0.01949167251586914
2020-12-07 09:37:14 MAC-ATI0414 [INFO] splits are as follow:
train    14933
val       3733
test         1
Name: split, dtype: int64
2020-12-07 09:37:14 MAC-ATI0414 [INFO] start creating a lookup table and convert characters to indices
2020-12-07 09:37:14 MAC-ATI0414 [INFO] -- create vocabulary


s2 padding:   0%|          | 0/14933 [00:00<?, ?it/s]

2020-12-07 09:37:14 MAC-ATI0414 [INFO] -- convert tokens to indices
2020-12-07 09:37:14 MAC-ATI0414 [INFO] -- create a lookup table for tokens
2020-12-07 09:37:14 MAC-ATI0414 [INFO] -- read list of characters from ../../dataset/characters_v001.vocab
2020-12-07 09:37:14 MAC-ATI0414 [INFO] -- Length of vocabulary: 7543





2020-12-07 09:37:15 MAC-ATI0414 [INFO] ******************************
2020-12-07 09:37:15 MAC-ATI0414 [INFO] **** (Bi-directional) GRU ****
2020-12-07 09:37:15 MAC-ATI0414 [INFO] ******************************
2020-12-07 09:37:15 MAC-ATI0414 [INFO] read inputs
2020-12-07 09:37:15 MAC-ATI0414 [INFO] create a two_parallel_rnns model
2020-12-07 09:37:15 MAC-ATI0414 [INFO] start fitting parameters
2020-12-07 09:37:15 MAC-ATI0414 [INFO] Number of batches: 467
2020-12-07 09:37:15 MAC-ATI0414 [INFO] Number of epochs: 5





Total number of params: 627303

two_parallel_rnns (
  (emb): Embedding(7543, 60), weights=((7543, 60),), parameters=452580
  (rnn_1): GRU(60, 60, num_layers=2, dropout=0.01, bidirectional=True), weights=((180, 60), (180, 60), (180,), (180,), (180, 60), (180, 60), (180,), (180,), (180, 120), (180, 60), (180,), (180,), (180, 120), (180, 60), (180,), (180,)), parameters=109440
  (attn_step1): Linear(in_features=120, out_features=60, bias=True), weights=((60, 120), (60,)), parameters=7260
  (attn_step2): Linear(in_features=60, out_features=1, bias=True), weights=((1, 60), (1,)), parameters=61
  (fc1): Linear(in_features=480, out_features=120, bias=True), weights=((120, 480), (120,)), parameters=57720
  (fc2): Linear(in_features=120, out_features=2, bias=True), weights=((2, 120), (2,)), parameters=242
)


2020-12-07 09:38:04 MAC-ATI0414 [INFO] 12/07/2020_09:38:04 -- Epoch: 1/5; Train; loss: 0.197; acc: 0.918; precision: 0.906, recall: 0.937, macrof1: 0.918, weightedf1: 0.918


2020-12-07 09:38:06 MAC-ATI0414 [INFO] 12/07/2020_09:38:06 -- Epoch: 1/5; Valid; loss: 1.168; acc: 0.717; precision: 0.648, recall: 0.975, macrof1: 0.693, weightedf1: 0.695
2020-12-07 09:38:06 MAC-ATI0414 [INFO] saving the model


2020-12-07 09:38:56 MAC-ATI0414 [INFO] 12/07/2020_09:38:56 -- Epoch: 2/5; Train; loss: 0.065; acc: 0.982; precision: 0.989, recall: 0.977, macrof1: 0.982, weightedf1: 0.982


2020-12-07 09:38:58 MAC-ATI0414 [INFO] 12/07/2020_09:38:58 -- Epoch: 2/5; Valid; loss: 0.758; acc: 0.783; precision: 0.712, recall: 0.966, macrof1: 0.774, weightedf1: 0.775
2020-12-07 09:38:58 MAC-ATI0414 [INFO] saving the model


2020-12-07 09:39:46 MAC-ATI0414 [INFO] 12/07/2020_09:39:46 -- Epoch: 3/5; Train; loss: 0.049; acc: 0.987; precision: 0.992, recall: 0.983, macrof1: 0.987, weightedf1: 0.987


2020-12-07 09:39:48 MAC-ATI0414 [INFO] 12/07/2020_09:39:48 -- Epoch: 3/5; Valid; loss: 1.643; acc: 0.690; precision: 0.626, recall: 0.974, macrof1: 0.658, weightedf1: 0.661
2020-12-07 09:39:48 MAC-ATI0414 [INFO] saving the model (early stopped) with least valid loss (checkpoint: 2) at ./models/BL_model/BL_model.model
2020-12-07 09:39:48 MAC-ATI0414 [INFO] saving the model
2020-12-07 09:39:48 MAC-ATI0414 [INFO] Early stopping at epoch: 3, selected epoch: 2




User time: 153.4943


## Finetune a pretrained model

In [2]:
# from DeezyMatch import finetune as dm_finetune

# # fine-tune a pretrained model stored at pretrained_model_path and pretrained_vocab_path 
# dm_finetune(input_file_path="./inputs/input_dfm_demo2.yaml", 
#             dataset_path="../../dataset/BL_IAMS_geonames.tsv", 
#             model_name="ft_BL_model",
#             pretrained_model_path="./models/BL_model/BL_model.model", 
#             pretrained_vocab_path="./models/BL_model/BL_model.vocab")

## Model inference

In [3]:
from DeezyMatch import inference as dm_inference

# model inference using a model stored at pretrained_model_path and pretrained_vocab_path 
dm_inference(input_file_path="./inputs/input_dfm_demo2.yaml",
             dataset_path="../../dataset/BL_IAMS_geonames.tsv", 
             pretrained_model_path="./models/BL_model/BL_model.model", 
             pretrained_vocab_path="./models/BL_model/BL_model.vocab")

2020-12-07 09:40:35 MAC-ATI0414 [INFO] read input file: ./inputs/input_dfm_demo2.yaml
2020-12-07 09:40:35 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:40:35 MAC-ATI0414 [INFO] pytorch will use: cpu
2020-12-07 09:40:35 MAC-ATI0414 [INFO] read CSV file: ../../dataset/BL_IAMS_geonames.tsv
2020-12-07 09:40:35 MAC-ATI0414 [INFO] number of labels, True: 9532 and False: 9135


s2 padding:   0%|          | 0/18667 [00:00<?, ?it/s]

2020-12-07 09:40:35 MAC-ATI0414 [INFO] skipping 0 lines


2020-12-07 09:40:45 MAC-ATI0414 [INFO] 12/07/2020_09:40:45 -- Epoch: 0/0; Test; loss: 0.195; acc: 0.945; precision: 0.920, recall: 0.977, macrof1: 0.945, weightedf1: 0.945
--- 9.397141933441162 seconds ---


## Generate query vectors

In [4]:
from DeezyMatch import inference as dm_inference

# generate vectors for queries (specified in dataset_path) 
# using a model stored at pretrained_model_path and pretrained_vocab_path 
dm_inference(input_file_path="./inputs/input_dfm_demo2.yaml",
            dataset_path="../../query_scenarios/ukcounties_queries.txt", 
            pretrained_model_path="./models/BL_model/BL_model.model", 
            pretrained_vocab_path="./models/BL_model/BL_model.vocab",
            inference_mode="vect",
            scenario="queries/demo2")

2020-12-07 09:40:56 MAC-ATI0414 [INFO] read input file: ./inputs/input_dfm_demo2.yaml
2020-12-07 09:40:57 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:40:57 MAC-ATI0414 [INFO] pytorch will use: cpu
2020-12-07 09:40:57 MAC-ATI0414 [INFO] read CSV file: ../../query_scenarios/ukcounties_queries.txt
2020-12-07 09:40:57 MAC-ATI0414 [INFO] number of labels, True: 0 and False: 35
2020-12-07 09:40:57 MAC-ATI0414 [INFO] skipping 0 lines
2020-12-07 09:40:57 MAC-ATI0414 [INFO] save test-data-class: /Users/khosseini/myJobs/ATI/Projects/2019/LaNC-workshop/deezymatch/demos/demo2_toy_dataset/queries/demo2/dataframe.df


--- 0.09249496459960938 seconds ---


## Generate candidate vectors

In [5]:
from DeezyMatch import inference as dm_inference

# generate vectors for candidates (specified in dataset_path) 
# using a model stored at pretrained_model_path and pretrained_vocab_path 
dm_inference(input_file_path="./inputs/input_dfm_demo2.yaml",
            dataset_path="../../candidate_scenarios/ukcounties_candidates.txt", 
            pretrained_model_path="./models/BL_model/BL_model.model", 
            pretrained_vocab_path="./models/BL_model/BL_model.vocab",
            inference_mode="vect",
            scenario="candidates/demo2")

2020-12-07 09:41:06 MAC-ATI0414 [INFO] read input file: ./inputs/input_dfm_demo2.yaml
2020-12-07 09:41:06 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:41:06 MAC-ATI0414 [INFO] pytorch will use: cpu
2020-12-07 09:41:06 MAC-ATI0414 [INFO] read CSV file: ../../candidate_scenarios/ukcounties_candidates.txt
2020-12-07 09:41:06 MAC-ATI0414 [INFO] number of labels, True: 0 and False: 94
2020-12-07 09:41:06 MAC-ATI0414 [INFO] skipping 0 lines
2020-12-07 09:41:06 MAC-ATI0414 [INFO] save test-data-class: /Users/khosseini/myJobs/ATI/Projects/2019/LaNC-workshop/deezymatch/demos/demo2_toy_dataset/candidates/demo2/dataframe.df


--- 0.10760092735290527 seconds ---


## Assembling queries vector representations

In [6]:
from DeezyMatch import combine_vecs

# combine vectors stored in queries/test and save them in combined/queries_test
combine_vecs(rnn_passes=['fwd', 'bwd'], 
             input_scenario='queries/demo2', 
             output_scenario='combined/queries_demo2', 
             print_every=10)

2020-12-07 09:41:14 MAC-ATI0414 [INFO] read input file: queries/demo2/input_dfm_demo2.yaml
2020-12-07 09:41:14 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:41:14 MAC-ATI0414 [INFO] pytorch will use: cpu


-- Combine vectors
Reading vectors from queries/demo2/embeddings/rnn_fwd*
0000000 queries/demo2/embeddings/rnn_fwd_0


-- Combine IDs
0000000 queries/demo2/embeddings/rnn_indxs_0



-- Combine vectors
Reading vectors from queries/demo2/embeddings/rnn_bwd*
0000000 queries/demo2/embeddings/rnn_bwd_0


-- Combine IDs
0000000 queries/demo2/embeddings/rnn_indxs_0

--- 239.859530210495 seconds ---


## Assembling candidates vector representations

In [7]:
from DeezyMatch import combine_vecs

# combine vectors stored in candidates/test and save them in combined/candidates_test
combine_vecs(rnn_passes=['fwd', 'bwd'], 
             input_scenario='candidates/demo2', 
             output_scenario='combined/candidates_demo2', 
             print_every=10)

2020-12-07 09:41:15 MAC-ATI0414 [INFO] read input file: candidates/demo2/input_dfm_demo2.yaml
2020-12-07 09:41:15 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:41:15 MAC-ATI0414 [INFO] pytorch will use: cpu


-- Combine vectors
Reading vectors from candidates/demo2/embeddings/rnn_fwd*
0000000 candidates/demo2/embeddings/rnn_fwd_0


-- Combine IDs
0000000 candidates/demo2/embeddings/rnn_indxs_0



-- Combine vectors
Reading vectors from candidates/demo2/embeddings/rnn_bwd*
0000000 candidates/demo2/embeddings/rnn_bwd_0


-- Combine IDs
0000000 candidates/demo2/embeddings/rnn_indxs_0

--- 240.6270911693573 seconds ---


## Candidate Ranker

In [8]:
from DeezyMatch import candidate_ranker

# Select candidates based on L2-norm distance (aka faiss distance):
# find candidates from candidate_scenario 
# for queries specified in query_scenario
candidates_pd = \
    candidate_ranker(query_scenario="./combined/queries_demo2",
                     candidate_scenario="./combined/candidates_demo2", 
                     ranking_metric="faiss", 
                     selection_threshold=100., 
                     num_candidates=5, 
                     search_size=5, 
                     output_path="ranker_results/candidates_deezymatch_demo2", 
                     pretrained_model_path="./models/BL_model/BL_model.model", 
                     pretrained_vocab_path="./models/BL_model/BL_model.vocab")

2020-12-07 09:41:20 MAC-ATI0414 [INFO] read input file: ./combined/candidates_demo2/input_dfm_demo2.yaml
2020-12-07 09:41:20 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:41:20 MAC-ATI0414 [INFO] pytorch will use: cpu
Is faiss_id_candis already trained? True
=========== Start the search for 0 E Ayrshire


ID: 1/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 1 North Lanarcshire


ID: 2/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 2

 N. Yorksh


ID: 3/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 3 N.amptonshire


...

ID: 31/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 31 Wiltsh


ID: 32/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 32 Worchestershire


ID: 33/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 33 Worcs


ID: 34/35 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 34 Tyne


ID: 35/35 -- Number of found candidates so far: 5, searched: 5
TOTAL TIME: 2.6396830081939697


In [9]:
candidates_pd

,query,pred_score,1-pred_score,faiss_distance,cosine_dist,candidate_original_ids,query_original_id,num_all_searches
id,,,,,,,,
0,E Ayrshire,"{'Eilean Siar': 0.7194, 'Pembrokeshire': 0.539...","{'Eilean Siar': 0.28059999999999996, 'Pembroke...","{'Eilean Siar': 13.675, 'Pembrokeshire': 17.34...","{'Eilean Siar': 0.4136, 'Pembrokeshire': 0.462...","{'Eilean Siar': 68, 'Pembrokeshire': 48, 'East...",0,5
1,North Lanarcshire,"{'North Lanarkshire': 0.9899, 'North Ayrshire'...","{'North Lanarkshire': 0.010099999999999998, 'N...","{'North Lanarkshire': 0.1857, 'North Ayrshire'...","{'North Lanarkshire': 0.004, 'North Ayrshire':...","{'North Lanarkshire': 77, 'North Ayrshire': 76...",1,5
2,N. Yorksh,"{'North Yorkshire': 0.9864, 'Oxfordshire': 0.9...","{'North Yorkshire': 0.013599999999999945, 'Oxf...","{'North Yorkshire': 7.6954, 'Oxfordshire': 9.4...","{'North Yorkshire': 0.126, 'Oxfordshire': 0.16...","{'North Yorkshire': 24, 'Oxfordshire': 28, 'No...",2,5
3,N.amptonshire,"{'Northamptonshire': 0.9931, 'Pembrokeshire': ...","{'Northamptonshire': 0.006900000000000017, 'Pe...","{'Northamptonshire': 2.558, 'Pembrokeshire': 2...","{'Northamptonshire': 0.0537, 'Pembrokeshire': ...","{'Northamptonshire': 25, 'Pembrokeshire': 48, ...",3,5
4,Northumbld,"{'Northumberland': 0.9903, 'Fermanagh': 0.9826...","{'Northumberland': 0.009700000000000042, 'Ferm...","{'Northumberland': 1.6716, 'Fermanagh': 11.603...","{'Northumberland': 0.0249, 'Fermanagh': 0.1883...","{'Northumberland': 26, 'Fermanagh': 91, 'Orkne...",4,5
5,Northumberl,"{'Northumberland': 0.9959, 'North Ayrshire': 0...","{'Northumberland': 0.0040999999999999925, 'Nor...","{'Northumberland': 4.9383, 'North Ayrshire': 6...","{'Northumberland': 0.0824, 'North Ayrshire': 0...","{'Northumberland': 26, 'North Ayrshire': 76, '...",5,5
6,Nottingham shire,"{'Nottinghamshire': 0.9908, 'North Ayrshire': ...","{'Nottinghamshire': 0.009199999999999986, 'Nor...","{'Nottinghamshire': 0.1109, 'North Ayrshire': ...","{'Nottinghamshire': 0.0025, 'North Ayrshire': ...","{'Nottinghamshire': 27, 'North Ayrshire': 76, ...",6,5
7,Orkneys,"{'Renfrewshire': 0.9978, 'Oxfordshire': 0.9989...","{'Renfrewshire': 0.0021999999999999797, 'Oxfor...","{'Renfrewshire': 7.6993, 'Oxfordshire': 8.4693...","{'Renfrewshire': 0.1478, 'Oxfordshire': 0.1603...","{'Renfrewshire': 80, 'Oxfordshire': 28, 'Moray...",7,5
8,Pembrockshire,"{'Pembrokeshire': 0.9907, 'North Ayrshire': 0....","{'Pembrokeshire': 0.009299999999999975, 'North...","{'Pembrokeshire': 0.0954, 'North Ayrshire': 3....","{'Pembrokeshire': 0.002, 'North Ayrshire': 0.0...","{'Pembrokeshire': 48, 'North Ayrshire': 76, 'N...",8,5


## Candidate ranking on-the-fly

In [10]:
from DeezyMatch import candidate_ranker

# Ranking on-the-fly
# find candidates from candidate_scenario 
# for queries specified by the `query` argument
candidates_pd = \
    candidate_ranker(query=["lincoln", "warwick", "warwickz"],
                     candidate_scenario="./combined/candidates_demo2", 
                     ranking_metric="faiss", 
                     selection_threshold=100., 
                     num_candidates=5, 
                     search_size=5, 
                     output_path="ranker_results/candidates_deezymatch_demo2", 
                     pretrained_model_path="./models/BL_model/BL_model.model", 
                     pretrained_vocab_path="./models/BL_model/BL_model.vocab")

2020-12-07 09:41:41 MAC-ATI0414 [INFO] read input file: ./combined/candidates_demo2/input_dfm_demo2.yaml
2020-12-07 09:41:41 MAC-ATI0414 [INFO] GPU was requested but not available.
2020-12-07 09:41:41 MAC-ATI0414 [INFO] pytorch will use: cpu
2020-12-07 09:41:41 MAC-ATI0414 [INFO] save test-data-class: tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/dataframe.df




-- Combine vectors
Reading vectors from tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/embeddings/rnn_fwd*
0000000 tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/embeddings/rnn_fwd_0


-- Combine IDs
0000000 tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/embeddings/rnn_indxs_0



-- Combine vectors
Reading vectors from tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/embeddings/rnn_bwd*
0000000 tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/embeddings/rnn_bwd_0


-- Combine IDs
0000000 tmp_561a9e10-cce8-490b-b8d6-3fc0f3c36bd1/query/embeddings/rnn_indxs_0

--- 266.55969405174255 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 lincoln


ID: 1/3 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 1 warwick


ID: 2/3 -- Number of found candidates so far: 5, searched: 5
=========== Start the search for 2 warwickz


ID: 3/3 -- Number of found candidates so far: 5, searched: 5
TOTAL TIME: 0.33826494216918945


In [11]:
candidates_pd

,query,pred_score,1-pred_score,faiss_distance,cosine_dist,candidate_original_ids,query_original_id,num_all_searches
id,,,,,,,,
0,lincoln,"{'Lincolnshire': 0.9974, 'Clackmannanshire': 0...","{'Lincolnshire': 0.0026000000000000467, 'Clack...","{'Lincolnshire': 7.1052, 'Clackmannanshire': 1...","{'Lincolnshire': 0.1034, 'Clackmannanshire': 0...","{'Lincolnshire': 21, 'Clackmannanshire': 61, '...",0,5
1,warwick,"{'Warwickshire': 0.9976, 'Wiltshire': 0.9985, ...","{'Warwickshire': 0.0023999999999999577, 'Wilts...","{'Warwickshire': 5.7448, 'Wiltshire': 13.742, ...","{'Warwickshire': 0.1339, 'Wiltshire': 0.3037, ...","{'Warwickshire': 36, 'Wiltshire': 41, 'Falkirk...",1,5
2,warwickz,"{'Warwickshire': 0.9967, 'Wiltshire': 0.9983, ...","{'Warwickshire': 0.0032999999999999696, 'Wilts...","{'Warwickshire': 5.0446, 'Wiltshire': 12.6466,...","{'Warwickshire': 0.1208, 'Wiltshire': 0.2853, ...","{'Warwickshire': 36, 'Wiltshire': 41, 'Flintsh...",2,5
